In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import ESMF as E

import importlib
import glob
import copy

import xyp_plot as xyp
import scripGen as SG
import esmfRegrid as erg


importlib.reload( erg )
importlib.reload(SG)

# Set-up regridding machinery

In [ ]:
# Get default CAM Topo bottom BCs for this grid
bnd_topo = \
'/glade/p/cgd/amp/juliob/bndtopo/latest/ne30pg3_gmted2010_modis_bedmachine_nc3000_Laplace0100_20230105.nc'
dsTopo_CAM=xr.open_dataset( bnd_topo )
PHIS_CAM = dsTopo_CAM['PHIS']
phis_CAM = PHIS_CAM.values

In [ ]:
# Scrip file for ERA5 created by ERA5scrip.ipynb
ERA5scrip = '/glade/work/juliob/ERA5-proc/ERA5interp/grids/ERA5_640x1280_scrip.nc'

src_scrip=ERA5scrip
src_type='grid'

In [ ]:
# ------- CAM SE ne30pg3 Scrip file
scripdir='/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
ne30scrip  = scripdir +  "ne30pg3_scrip_170611.nc"

dst_scrip=ne30scrip
dst_type='mesh'

In [ ]:
griddir = "/glade/work/juliob/ERA5-proc/ERA5interp/grids/"
wgts_file_Con = griddir + "ERA5_ne30pg3_Conserv_wgts.nc"

# Make object for Conservative regridding from ERA5
# grid to CAM target. Scrip files need to be provided even 
# when a weight file is used
regrd, srcf, dstf = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                write_weights = False ,
                                read_weights = True ,
                                weights_file = wgts_file_Con )

In [ ]:
# Read in ERA5 topogrpahy
eratopof='/glade/work/juliob/ERA5-proc/ERA5interp/phis/ERA5_phis.nc'
eratopo=xr.open_dataset( eratopof )
phis_ERA=eratopo['Z_GDS4_SFC'].values

# Regrid ERA5 topo and read in CAM target topo

In [ ]:
ny,nx = np.shape(phis_ERA)
ncol=np.shape(phis_CAM)
#try function
#########################
phis_ERA_x_CAM = erg.HorzRG( aSrc = phis_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcShape=[ny,nx], 
                              dstShape=[ncol] , 
                              srcGridkey='yx', 
                              dstGridkey='c' )


# Regrid ERA5 data

In [ ]:
era5dir = "/glade/collections/rda/data/ds633.6/e5.oper.an.ml/"
wrkdir=era5dir+"202211/"
tfile = wrkdir + 'e5.oper.an.ml.0_5_0_0_0_t.regn320sc.2022110100_2022110105.nc'
spfile= wrkdir + 'e5.oper.an.ml.128_134_sp.regn320sc.2022110100_2022110105.nc'

In [ ]:
dsT_ERA   = xr.open_dataset( tfile )
dsPS_ERA  = xr.open_dataset( spfile )


In [ ]:
print( list(dsT_ERA.variables) )
print( list(dsPS_ERA.variables) )


In [ ]:
ps_ERA = dsPS_ERA['SP'].values
print( np.shape(ps_ERA))
nt,ny,nx = np.shape(ps_ERA)

In [ ]:
print(ncol[0])

In [ ]:
#try function
#########################
ps_ERA_x_CAM    = erg.HorzRG( aSrc = ps_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcShape=[nt,ny,nx], 
                              dstShape=[ncol[0]] , 
                              srcGridkey='tyx', 
                              dstGridkey='c' )


In [ ]:
lon_CAM=dsTopo_CAM['lon'].values
lat_CAM=dsTopo_CAM['lat'].values

#plt.figure(figsize=(20, 20))
fig,ax = plt.subplots(2,2,figsize=(15,7),squeeze=False)


j,i=0,0
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, phis_ERA_x_CAM )
plt.colorbar(co,ax=ax[j,i])
j,i=0,1
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, ps_ERA_x_CAM[0,:] )
plt.colorbar(co,ax=ax[j,i])
j,i=1,0
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, ps_ERA_x_CAM[1,:] )
plt.colorbar(co,ax=ax[j,i])
j,i=1,1
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, ps_ERA_x_CAM[5,:] )
plt.colorbar(co,ax=ax[j,i])


# Now try a full horizontal regridding of 3(4)-D ERA T

In [ ]:
te_ERA = dsT_ERA['T'].values
nt,nz,ny,nx = np.shape(te_ERA)


In [ ]:
#try function
#########################
te_ERA_x_CAM    = erg.HorzRG( aSrc = te_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcShape=[nt,nz,ny,nx], 
                              dstShape=[ncol[0]] , 
                              srcGridkey='tzyx', 
                              dstGridkey='c' )


In [ ]:
lon_CAM=dsTopo_CAM['lon'].values
lat_CAM=dsTopo_CAM['lat'].values

#plt.figure(figsize=(20, 20))
fig,ax = plt.subplots(2,2,figsize=(15,7),squeeze=False)


j,i=0,0
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, te_ERA_x_CAM[0,20,:] )
plt.colorbar(co,ax=ax[j,i])
j,i=0,1
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, te_ERA_x_CAM[0,40,:] )
plt.colorbar(co,ax=ax[j,i])
j,i=1,0
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, te_ERA_x_CAM[0,80,:] )
plt.colorbar(co,ax=ax[j,i])
j,i=1,1
co = ax[j,i].tricontourf( lon_CAM, lat_CAM, te_ERA_x_CAM[0,100,:] )
plt.colorbar(co,ax=ax[j,i])


In [ ]:
print(len(np.shape(dstf.data)))

In [ ]:
print(len(np.shape(te_ERA)))